# Part 1: Network Training

## Step0: Import Package & Hyperparameter Configuration

In [53]:
# 清空所有變數
%reset -f
# # 強制 Python 回收記憶體
# import gc
# gc.collect()

### Package


In [54]:
import os
import torch
import numpy as np
import random
import torch.nn as nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
import time
from datetime import datetime
import json
import pandas as pd
import optuna

try:
    os.chdir(os.path.dirname(os.path.abspath(__file__)))
except NameError:
    print("Notebook 環境，跳過切換目錄")

Notebook 環境，跳過切換目錄


### Hyperparameter Config

In [55]:
# %%
# Unified Hyperparameter Configuration
class Config:
    SEED = 1
    NUM_EPOCHS = 1500
    BATCH_SIZE = 256
    LEARNING_RATE = 0.002  #論文提供
    LR_SCHEDULER_GAMMA = 0.99  #論文提供
    DECAY_EPOCH = 200
    EARLY_STOPPING_PATIENCE = 100
    HIDDEN_SIZE = 30
    OPERATOR_SIZE = 30
    MAXOUT_H = 1


# Reproducibility
random.seed(Config.SEED)
np.random.seed(Config.SEED)
torch.manual_seed(Config.SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


### Material & Number of Data

In [56]:
material = "CH467160"
fix_way = "uesed_for_PFC_test"
note = "optuna_search_1"
note_detail = "找 BATCH_SIZE、學習率、隱藏層大小、運算子大小的最佳組合"
downsample = 1024
save_figure = False
timestamp = datetime.now().strftime("%Y%m%d")

# 訓練情況況
plot_interval = 150
train_show_sample = 1

result_dir = os.path.join("results",
                          f"{timestamp}_{fix_way}_{material}_{note}")
os.makedirs(result_dir, exist_ok=True)

# 定義保存模型的路徑
model_save_dir = result_dir
model_save_path = os.path.join(
    model_save_dir, f"{material}_{fix_way}_{note}_{timestamp}.pt")  # 定義模型保存檔名

figure_save_base_path = result_dir

# Select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Step1: Data processing and data loader generate 

In [57]:
# %% Preprocess data into a data loader
def get_dataloader(data_B,
                   data_F,
                   data_T,
                   data_H,
                   data_N,
                   data_Hdc,
                   data_Duty_P,
                   data_Duty_N,
                   data_Pcv,
                   global_B_max,
                   global_H_max,
                   batch_size,
                   operator_size,
                   n_init=16):

    # Data pre-process

    # ── 0. 全域設定/降階設定 ──────────────────────────────
    eps = 1e-8  # 防止除以 0
    if downsample == 1024:
        seq_length = 1024  # 單筆波形點數 (不再 down-sample)
    else:
        seq_length = downsample
        cols = np.linspace(0, 1023, seq_length, dtype=int)
        data_B = data_B[:, cols]
        data_H = data_H[:, cols]

    # ── 1. 波形拼接 (補 n_init 點作初始磁化) ────
    data_length = seq_length + n_init
    data_B = np.hstack((data_B[:, -n_init:], data_B))  # (batch, data_length)
    data_H = np.hstack((data_H[:, -n_init:], data_H))

    print("B shape:", data_B.shape)
    print("H shape:", data_H.shape)
    print("F shape:", data_F.shape)
    print("T shape:", data_T.shape)
    print("Hdc shape:", data_Hdc.shape)
    print("N shape:", data_N.shape)
    print("Duty Pos shape:", data_Duty_P.shape)
    print("Duty Neg shape:", data_Duty_N.shape)
    print("Pcv shape:", data_Pcv.shape)

    # ── 2. 轉成 Tensor ───────────────────────────
    B = torch.from_numpy(data_B).view(-1, data_length, 1).float()  # (B,N,1)
    H = torch.from_numpy(data_H).view(-1, data_length, 1).float()
    F = torch.log10(torch.from_numpy(data_F).view(-1, 1).float())  # 純量
    T = torch.from_numpy(data_T).view(-1, 1).float()
    Hdc = torch.from_numpy(data_Hdc).view(-1, 1).float()
    N = torch.from_numpy(data_N).view(-1, 1).float()
    Duty_P = torch.from_numpy(data_Duty_P).view(-1, 1).float()
    Duty_N = torch.from_numpy(data_Duty_N).view(-1, 1).float()
    Pcv = torch.log10(torch.from_numpy(data_Pcv).view(-1, 1).float())

    # ── 3. 每筆樣本各自找最大幅值 (per-profile scale) ─
    # scale_B = torch.max(torch.abs(B), dim=1,
    #                     keepdim=True).values + eps  # (B,1,1)
    # scale_H = torch.max(torch.abs(H), dim=1, keepdim=True).values + eps

    # ── 4. 先計算導數，再除以 scale_B ─────────────
    dB = torch.diff(B, dim=1, prepend=B[:, :1])
    dB_dt = dB * (seq_length * F.view(-1, 1, 1))  # 真實斜率
    # d2B = torch.diff(dB, dim=1, prepend=dB[:, :1])
    # d2B_dt = d2B * (seq_length * F.view(-1, 1, 1))

    # ── 5. 形成模型輸入 (已經縮放到 [-1,1]) ────────
    # in_B = B / scale_B
    # out_H = H / scale_H  # 預測目標
    # in_dB_dt = dB_dt / scale_B
    # 後續發現d2B無改善準確度(可能要多波形種類才有效幫助)，先以輸入0代入
    # in_d2B_dt = d2B_dt / scale_B

    # *修正成使用全域最大幅值 (ver.250806)
    in_B = B / global_B_max
    out_H = H / global_H_max
    in_dB_dt = dB_dt / global_B_max
    in_d2B_dt = torch.zeros_like(in_dB_dt)

    # ── 6. 純量特徵：計算 z-score 參數 ─────────────
    def safe_mean_std(tensor, eps=1e-8):
        m = torch.mean(tensor).item()
        s = torch.std(tensor).item()
        return [m, 1.0 if s < eps else s]

    #  Compute normalization parameters (均值 & 標準差)**
    norm = [
        safe_mean_std(F),
        safe_mean_std(T),
        safe_mean_std(Hdc),
        safe_mean_std(N),
        safe_mean_std(Pcv)
    ]

    # # 用來做test固定標準化參數的
    # print("0.F, 1.T, 2.Hdc, 3.N, 4.Pcv")
    # material_name = f"{material}"
    # print(f'"{material_name}": [')
    # for param in norm:
    #     print(f"    {param},")
    # print("]")

    # Data Normalization
    in_F = (F - norm[0][0]) / norm[0][1]  # F
    in_T = (T - norm[1][0]) / norm[1][1]  # T
    in_Hdc = (Hdc - norm[2][0]) / norm[2][1]  # Hdc
    in_N = (N - norm[3][0]) / norm[3][1]  # N
    in_Pcv = (Pcv - norm[4][0]) / norm[4][1]  # Pcv
    in_Duty_P = Duty_P  # Duty Pos
    in_Duty_N = Duty_N  # Duty Neg

    # #   → 方便推論復原，保留 scale_B, scale_H 當作額外純量
    # aux_features = torch.cat(
    #     (in_F, in_T, in_Hdc, in_N, in_Duty_P, in_Duty_N, in_Pcv,
    #      scale_B.squeeze(-1), scale_H.squeeze(-1)),
    #     dim=1)

    # ── 7. 產生初始 Preisach operator 狀態 s0 ──────
    max_B, _ = torch.max(in_B, dim=1)
    min_B, _ = torch.min(in_B, dim=1)
    # s0 = get_operator_init(in_B[:, 0] - dB[:, 0] / scale_B.squeeze(-1),
    #                        dB / scale_B, max_B, min_B)

    s0 = get_operator_init(in_B[:, 0] - dB[:, 0] / global_B_max.squeeze(-1),
                           dB / global_B_max,
                           max_B,
                           min_B,
                           operator_size=operator_size)

    # ── 8. 組合 Dataset ───────────────────────────
    # wave_inputs = torch.cat(
    #     (
    #         in_B,  # ① B
    #         dB / scale_B,  # ② ΔB
    #         in_dB_dt,  # ③ dB/dt
    #         in_d2B_dt),
    #     dim=2)  # ④ d²B/dt²   → (B,L,4)

    # amps = torch.cat((scale_B.squeeze(-1), scale_H.squeeze(-1)),
    #                 dim=1)  # (B,2)

    wave_inputs = torch.cat(
        (
            in_B,  # ① B
            dB / global_B_max,  # ② ΔB
            in_dB_dt,  # ③ dB/dt
            in_d2B_dt),
        dim=2)  # ④ d²B/dt²   → (B,L,4)

    aux_features = torch.cat((in_F, in_T, in_Hdc, in_N, in_Duty_P, in_Duty_N),
                             dim=1)  # (B,4)

    amp_B = torch.full((len(B), 1), global_B_max, dtype=torch.float32)
    amp_H = torch.full((len(B), 1), global_H_max, dtype=torch.float32)
    amps = torch.cat((amp_B, amp_H), dim=1)  # 仍給 RNN2 用

    # 這裡把 Pcv（已 z-score）單獨拿出來當另一個 label
    target_Pcv = in_Pcv  # (B,1)

    full_dataset = torch.utils.data.TensorDataset(
        wave_inputs,  # 0  → 模型序列輸入
        aux_features,  # 1  → 4 個純量
        amps,  # 2  → 幅值係數
        s0,  # 3  → Preisach 初始狀態
        out_H,  # 4  → 目標 H  (已 scale_H)
        target_Pcv)  # 5  → 目標 Pcv (已 z-score)

    # ── 9. Train / Valid split & DataLoader ───────
    train_size = int(0.8 * len(full_dataset))
    valid_size = len(full_dataset) - train_size
    train_set, valid_set = torch.utils.data.random_split(
        full_dataset, [train_size, valid_size],
        generator=torch.Generator().manual_seed(Config.SEED))

    train_loader = torch.utils.data.DataLoader(train_set,
                                               batch_size=batch_size,
                                               shuffle=True,
                                               num_workers=0,
                                               pin_memory=True,
                                               collate_fn=filter_input)

    valid_loader = torch.utils.data.DataLoader(valid_set,
                                               batch_size=batch_size,
                                               shuffle=False,
                                               num_workers=0,
                                               pin_memory=True,
                                               collate_fn=filter_input)

    return train_loader, valid_loader, norm


# %% Predict the operator state at t0
def get_operator_init(B1,
                      dB,
                      Bmax,
                      Bmin,
                      max_out_H=Config.MAXOUT_H,
                      operator_size=Config.OPERATOR_SIZE):
    """Compute the initial state of hysteresis operators"""
    s0 = torch.zeros((dB.shape[0], operator_size))
    operator_thre = torch.from_numpy(
        np.linspace(max_out_H / operator_size, max_out_H,
                    operator_size)).view(1, -1)

    for i in range(dB.shape[0]):
        for j in range(operator_size):
            r = operator_thre[0, j]
            if (Bmax[i] >= r) or (Bmin[i] <= -r):
                if dB[i, 0] >= 0:
                    if B1[i] > Bmin[i] + 2 * r:
                        s0[i, j] = r
                    else:
                        s0[i, j] = B1[i] - (r + Bmin[i])
                else:
                    if B1[i] < Bmax[i] - 2 * r:
                        s0[i, j] = -r
                    else:
                        s0[i, j] = B1[i] + (r - Bmax[i])
    return s0


def filter_input(batch):
    inputs, features, amps, s0, target_H, target_Pcv = zip(*batch)

    inputs = torch.stack(inputs)
    features = torch.stack(features)
    amps = torch.stack(amps)
    s0 = torch.stack(s0)
    target_H = torch.stack(target_H)[:, -downsample:, :]  # 保留全長
    target_Pcv = torch.stack(target_Pcv)  # (B,1)

    return inputs, features, amps, s0, target_H, target_Pcv


# 溫度頻率不變加入微小的 epsilon
def safe_mean_std(tensor, eps=1e-8):
    m_tensor = torch.mean(tensor)  # 還是 Tensor
    s_tensor = torch.std(tensor)  # 還是 Tensor

    m_val = m_tensor.item()  # 第一次轉成 float
    s_val = s_tensor.item()
    if s_val < eps:
        s_val = 1.0
    return [m_val, s_val]  # 直接回傳 float


## Step2: Define Network Structure

In [58]:
# %% Magnetization mechansim-determined neural network
"""
    Parameters:
    - hidden_size: number of eddy current slices (RNN neuron)
    - operator_size: number of operators
    - input_size: number of inputs (1.B 2.dB 3.dB/dt 4.d2B/dt)
    - var_size: number of supplenmentary variables (1.F 2.T 3.Hdc 4.N 5.Duty_P 6.Duty_N)        
    - output_size: number of outputs (1.H)
    
    只先把d2B/dt考量在EddyCell裡面
"""


class MMINet(nn.Module):

    def __init__(self,
                 norm,
                 hidden_size,
                 operator_size,
                 input_size=4,
                 var_size=6,
                 output_size=1):
        super().__init__()
        self.input_size = input_size
        self.var_size = var_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.operator_size = operator_size
        self.norm = norm

        self.rnn1 = StopOperatorCell(self.operator_size)
        self.dnn1 = nn.Linear(self.operator_size + self.var_size, 1)
        # var_size (F T Hdc N Duty_P Duty_N ) + 3 (B, dB/dt, d2B/dt)
        self.rnn2 = EddyCell(var_size + 3, self.hidden_size, output_size)
        self.dnn2 = nn.Linear(self.hidden_size, 1)
        self.rnn2_hx = None
        # var_size=6: 1.F 2.T 3.Hdc 4.N 5.Duty_P 6.Duty_N + 1 for P_prelim
        self.loss_mlp = nn.Sequential(nn.Linear(self.var_size + 1, 128),
                                      nn.ReLU(), nn.Linear(128, 64), nn.ReLU(),
                                      nn.Linear(64, 32), nn.ReLU(),
                                      nn.Linear(32, 1))

    def forward(self, x, var, amps, s0, n_init=16):
        """
        Parameters: 
        - x(batch,seq,input_size): Input features (1.B, 2.dB, 3.dB/dt)  
        - var(batch,var_size): Supplementary inputs (1.F 2.T 3.Hdc 4.N 5.Duty_P 6.Duty_N) 
        - s0(batch,1): Operator inital states
        """
        batch_size = x.size(0)  # Batch size
        seq_size = x.size(1)  # Ser
        self.rnn1_hx = s0

        # !Initialize DNN2 input (1.B 2.dB/dt 3.d2B)
        # x2 = torch.cat((x[:, :, 0:1], x[:, :, 2:3]), dim=2)
        # !選取 B, dB/dt, d2B/dt
        x2 = torch.cat((x[:, :, 0:1], x[:, :, 2:4]), dim=2)

        for t in range(seq_size):
            # RNN1 input (dB,state)
            self.rnn1_hx = self.rnn1(x[:, t, 1:2], self.rnn1_hx)

            # DNN1 input (rnn1_hx,F,T,Hdc,N)
            dnn1_in = torch.cat((self.rnn1_hx, var), dim=1)

            # H hysteresis prediction
            H_hyst_pred = self.dnn1(dnn1_in)

            # DNN2 input (B,dB/dt,T,F)
            rnn2_in = torch.cat((x2[:, t, :], var), dim=1)

            # Initialize second rnn state
            if t == 0:
                H_eddy_init = x[:, t, 0:1] - H_hyst_pred
                buffer = x.new_ones(x.size(0), self.hidden_size)
                self.rnn2_hx = Variable(
                    (buffer / torch.sum(self.dnn2.weight, dim=1)) *
                    H_eddy_init)

            #rnn2_in = torch.cat((rnn2_in,H_hyst_pred),dim=1)
            self.rnn2_hx = self.rnn2(rnn2_in, self.rnn2_hx)

            # H eddy prediction
            H_eddy = self.dnn2(self.rnn2_hx)

            # H total
            H_total = (H_hyst_pred + H_eddy).view(batch_size, 1,
                                                  self.output_size)
            if t == 0:
                output = H_total
            else:
                output = torch.cat((output, H_total), dim=1)

        H = (output[:, n_init:, :])

        amp_B = amps[:, 0:1]  # (batch,1)
        amp_H = amps[:, 1:2]  # (batch,1)
        B_amp = x[:, n_init:, 0:1] * amp_B.unsqueeze(1)
        H_amp = output[:, n_init:, :] * amp_H.unsqueeze(1)
        P_prelim = torch.trapz(H_amp, B_amp, axis=1) * (10**(
            var[:, 0:1] * self.norm[0][1] + self.norm[0][0]))
        Pcv_log = torch.log10(P_prelim.clamp(min=1e-12))
        Pcv = (Pcv_log - self.norm[4][0]) / self.norm[4][1]
        mlp_input = torch.cat((var, Pcv), dim=1)  # (batch, 5)
        s = self.loss_mlp(mlp_input)
        Pcv_mlp = Pcv + s

        return H, Pcv_mlp


class StopOperatorCell():

    def __init__(self, operator_size):
        self.operator_thre = torch.from_numpy(
            np.linspace(Config.MAXOUT_H / operator_size, Config.MAXOUT_H,
                        operator_size)).view(1, -1)

    def sslu(self, X):
        a = torch.ones_like(X)
        return torch.max(-a, torch.min(a, X))

    def __call__(self, dB, state):
        r = self.operator_thre.to(dB.device)
        output = self.sslu((dB + state) / r) * r
        return output.float()


class EddyCell(nn.Module):

    def __init__(self, input_size, hidden_size, output_size=1):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.x2h = nn.Linear(input_size, hidden_size, bias=False)
        self.h2h = nn.Linear(hidden_size, hidden_size, bias=False)

    def forward(self, x, hidden=None):
        hidden = self.x2h(x) + self.h2h(hidden)
        hidden = torch.sigmoid(hidden)
        return hidden


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

## Step3: Training the Model

### Load Dataset

In [59]:
# %%
def load_dataset(material, base_path="./Data/"):

    in_file1 = f"{base_path}{material}/train/B_Field.csv"
    in_file2 = f"{base_path}{material}/train/Frequency.csv"
    in_file3 = f"{base_path}{material}/train/Temperature.csv"
    in_file4 = f"{base_path}{material}/train/H_Field.csv"
    in_file5 = f"{base_path}{material}/train/Volumetric_Loss.csv"
    in_file6 = f"{base_path}{material}/train/Hdc.csv"
    in_file7 = f"{base_path}{material}/train/Turns.csv"
    in_file8 = f"{base_path}{material}/train/Duty_P.csv"
    in_file9 = f"{base_path}{material}/train/Duty_N.csv"

    data_B = np.genfromtxt(in_file1, delimiter=',')  # N x 1024
    data_F = np.genfromtxt(in_file2, delimiter=',')  # N x 1
    data_T = np.genfromtxt(in_file3, delimiter=',')  # N x 1
    data_H = np.genfromtxt(in_file4, delimiter=',')  # N x 1024
    data_Pcv = np.genfromtxt(in_file5, delimiter=',')  # N x 1
    data_Hdc = np.genfromtxt(in_file6, delimiter=',')  # N x 1
    data_N = np.genfromtxt(in_file7, delimiter=',')  # N x 1
    data_Duty_P = np.genfromtxt(in_file8, delimiter=',')  # N x 1
    data_Duty_N = np.genfromtxt(in_file9, delimiter=',')  # N x 1

    return data_B, data_F, data_T, data_H, data_Pcv, data_Hdc, data_N, data_Duty_P, data_Duty_N


### Train Logger

In [60]:
# class TrainLogger:

#     def __init__(self, exp_name, config_dict, result_dir):
#         self.exp_name = exp_name
#         self.result_dir = result_dir
#         self.config = config_dict
#         os.makedirs(self.result_dir, exist_ok=True)

#         self._save_config()
#         self._write_metadata()

#     def _save_config(self):
#         with open(os.path.join(self.result_dir, "config.json"), "w") as f:
#             json.dump(self.config, f, indent=2, ensure_ascii=False)

#     def _write_metadata(self):
#         metadata = {
#             "experiment_name": self.exp_name,
#             "timestamp": datetime.now().isoformat()
#         }
#         with open(os.path.join(self.result_dir, "meta.json"), "w") as f:
#             json.dump(metadata, f, indent=2)

#     def save_norm_params(self, norm, feature_names=["F", "T", "Hdc", "Pcv"]):
#         """
#         將標準化參數存成：
#         {
#           "CH467160": [
#              [mean_F, std_F],
#              [mean_T, std_T],
#              [mean_Hdc, std_Hdc],
#              [mean_N, std_N],
#              [mean_Pcv, std_Pcv],
#           ]
#         }
#         """
#         # 從 exp_name 前半段取出 material
#         material_key = self.exp_name.split('_')[0]

#         # 直接把 norm (list of [mean, std]) 當成 value
#         output = {material_key: norm}

#         # 寫檔
#         with open(os.path.join(self.result_dir, "norm_params.json"), "w") as f:
#             json.dump(output, f, indent=4, ensure_ascii=False)
#         print(
#             f"✅ Normalization parameters saved to {os.path.join(self.result_dir, 'norm_params.json')}"
#         )

#     def save_summary(self, best_epoch, best_val_loss, best_loss_H,
#                      best_loss_Pcv, model_save_path, elapsed):
#         summary = {
#             "exp_name": self.exp_name,
#             "timestamp": datetime.now().isoformat(),
#             "duration_sec": elapsed,
#             "config": self.config,
#             "best_model": {
#                 "path": model_save_path,
#                 "epoch": best_epoch,
#                 "val_loss": best_val_loss,
#                 "loss_H": best_loss_H,
#                 "loss_Pcv": best_loss_Pcv
#             },
#             "note": note,
#             "note detail": note_detail
#         }
#         with open(os.path.join(self.result_dir, "summary.json"), "w") as f:
#             json.dump(summary, f, indent=2, ensure_ascii=False)

### Train Code

In [61]:
def clamp_learning_rate(optimizer, min_lr=1e-5):
    for param_group in optimizer.param_groups:
        if param_group['lr'] < min_lr:
            param_group['lr'] = min_lr

In [62]:
def train_model(trial, config_dict, norm, train_loader, valid_loader):

    # 從傳入的 config_dict 取得超參數
    LEARNING_RATE = config_dict['LEARNING_RATE']
    LR_SCHEDULER_GAMMA = config_dict['LR_SCHEDULER_GAMMA']
    HIDDEN_SIZE = config_dict['HIDDEN_SIZE']
    OPERATOR_SIZE = config_dict['OPERATOR_SIZE']
    BATCH_SIZE = config_dict['BATCH_SIZE']
    DECAY_EPOCH = config_dict.get('DECAY_EPOCH', Config.DECAY_EPOCH)

    # model = MMINet(norm=norm).to(device)
    model = MMINet(norm, hidden_size=HIDDEN_SIZE,
                   operator_size=OPERATOR_SIZE).to(device)
    print("=== Start Train  ===")
    print(r"""
          
          
                                                    ⠀⠀⠀⠀⢀⡤⠖⠋⠉⠉⠉⠉⠙⠲⣦⣀⠀⠀⠀⠀⠀
                                                    ⠀⠀⠀⡴⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀
                                                    ⠀⠀⡼⢁⡠⢼⠁⠀⢱⢄⣀⠀⠀⠀⠀⠀⠎⢿⡄⠀⠀
                                                    ⠀⣸⠁⠀⣧⣼⠀⠀⣧⣼⠉⠀⠀⠀⠀⠀⠐⢬⣷⠀⠀
                                                    ⡼⣿⢀⠀⣿⡟⠀⠀⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⢹⣧⠀
                  我想畢業...                       ⣇⢹⠀⠁⠈⠀⠉⠃⠈⠃⠀⠀⠀⠀⠀⠀⠀⠀⡰⢸⡇
                                                    ⠙⢿⣧⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⣏⣈⣉⣤⠿⠁
                                                    ⠀⣠⣾⣿⠤⡀⠀⠀⠀⠀⠀⢀⣤⣶⣿⣿⣿⣿⣅⠀⠀
                                                    ⢰⣧⣿⣿⣿⣦⣉⡐⠒⠒⢲⣿⣿⣿⣿⣿⣿⣶⣿⣧⠀
                                                    ⠘⠿⢿⣿⣿⣿⡿⠿⠛⠿⠿⠿⣿⣿⣿⣿⣿⣿⡿⠟⠀
                                                    ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⠁⠀⠀⠀⠀

    """)
    print("Number of parameters: ", count_parameters(model))

    criterion_H = nn.MSELoss()
    criterion_Pcv = nn.MSELoss()

    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=DECAY_EPOCH,
                                                gamma=LR_SCHEDULER_GAMMA)

    # Loss 記錄
    best_val_loss = float('inf')
    best_val_loss_Pcv = float('inf')
    best_val_loss_H = float('inf')

    for epoch in range(Config.NUM_EPOCHS):

        alpha = (epoch + 1) / Config.NUM_EPOCHS
        model.train()
        train_loss = 0

        for inputs, features, amps, s0, target_H, target_Pcv in train_loader:

            inputs, features, amps, s0, target_H, target_Pcv = inputs.to(
                device), features.to(device), amps.to(device), s0.to(
                    device), target_H.to(device), target_Pcv.to(device)

            optimizer.zero_grad()

            with torch.autocast(device_type="cuda"):
                outputs_H, outputs_Pcv = model(inputs, features, amps,
                                               s0)  # 模型的輸出
                loss_H = criterion_H(outputs_H, target_H)  # 使用真實的 H(t) 計算損失
                loss_Pcv = criterion_Pcv(outputs_Pcv, target_Pcv)

                loss = (1 - alpha) * loss_H + alpha * loss_Pcv
                # alpha = 0.5

            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        scheduler.step()  # scheduler 更新
        clamp_learning_rate(optimizer)  # 避免learning rate掉到 0
        current_lr = optimizer.param_groups[0]['lr']
        # print(f"Epoch {epoch+1} | Learning Rate: {current_lr:.6f}")

        # ------------------------------vaildation------------------------------

        model.eval()
        val_loss = 0
        val_loss_H = 0.0
        val_loss_Pcv = 0.0

        with torch.no_grad():
            for inputs, features, amps, s0, target_H, target_Pcv in valid_loader:
                inputs, features, amps, s0, target_H, target_Pcv = inputs.to(
                    device), features.to(device), amps.to(device), s0.to(
                        device), target_H.to(device), target_Pcv.to(device)

                outputs_H, outputs_Pcv = model(inputs, features, amps,
                                               s0)  # 模型的輸出
                loss_H = criterion_H(outputs_H, target_H)  # 使用真實的 H(t) 計算損失
                loss_Pcv = criterion_Pcv(outputs_Pcv, target_Pcv)

                loss = (1 - alpha) * loss_H + alpha * loss_Pcv

                val_loss += loss.item()
                val_loss_H += loss_H.item()
                val_loss_Pcv += loss_Pcv.item()

        # 求驗證集平均
        val_loss_H /= len(valid_loader)
        val_loss_Pcv /= len(valid_loader)
        val_loss /= len(valid_loader)

        if val_loss_H < best_val_loss_H and val_loss_Pcv < best_val_loss_Pcv:
            best_val_loss = val_loss
            best_val_loss_H = val_loss_H
            best_val_loss_Pcv = val_loss_Pcv

        # <<< OPTUNA 變更點: Pruning (剪枝) >>>
        # 1. 報告中間結果給 Optuna
        trial.report(val_loss, epoch)
        # 2. 檢查是否應該提早停止這個試驗
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    # 訓練迴圈結束後，回傳這次試驗的最佳驗證損失
    return best_val_loss

### Start Train!!!

In [ ]:
def objective(trial):
    # --- 1. 定義要搜尋的超參數空間 ---
    # 我先選幾個關鍵的當範例，你可以自己增加或修改
    config_dict = {
        'LEARNING_RATE':
        trial.suggest_float('LEARNING_RATE', [0.01, 0.05, 0.1, 0.2]),
        'BATCH_SIZE':
        trial.suggest_categorical('BATCH_SIZE', 64, 256, step=64),
        'HIDDEN_SIZE':
        trial.suggest_int('HIDDEN_SIZE', 10, 40, step=10),
        'OPERATOR_SIZE':
        trial.suggest_int('OPERATOR_SIZE', 10, 40, step=10),
        'LR_SCHEDULER_GAMMA':
        trial.suggest_float('LR_SCHEDULER_GAMMA', 0.95, 0.999),
        'DECAY_EPOCH':
        trial.suggest_int('DECAY_EPOCH', 50, 500),
    }

    # --- 2. 準備數據 ---
    # 每次試驗都重新載入數據，確保獨立性
    data_B, data_F, data_T, data_H, data_Pcv, data_Hdc, data_N, data_Duty_P, data_Duty_N = load_dataset(
        material)
    GLOBAL_B_MAX = np.abs(data_B).max()
    GLOBAL_H_MAX = np.abs(data_H).max()

    # 這裡的 get_operator_init 可能也需要 operator_size
    # 我們需要在 get_dataloader 內部調用時傳入
    # 為了簡化，我們先在 get_dataloader 內部直接呼叫 get_operator_init 時固定或傳入
    # 我已在上面 get_dataloader 和 get_operator_init 做了修改

    train_loader, valid_loader, norm = get_dataloader(
        data_B,
        data_F,
        data_T,
        data_H,
        data_N,
        data_Hdc,
        data_Duty_P,
        data_Duty_N,
        data_Pcv,
        GLOBAL_B_MAX,
        GLOBAL_H_MAX,
        batch_size=config_dict['BATCH_SIZE'],  # 使用 Optuna 建議的 batch size
        operator_size=config_dict['OPERATOR_SIZE'])

    # --- 3. 執行訓練並取得結果 ---
    try:
        val_loss = train_model(trial, config_dict, norm, train_loader,
                               valid_loader)
    except RuntimeError as e:
        # 有時參數組合不好會導致 CUDA out of memory，這時我們告訴 Optuna 這次試驗失敗
        if "out of memory" in str(e):
            print(f"Trial {trial.number} failed with OOM. Pruning.")
            # 回傳一個很大的數字，Optuna 就知道這是不好的參數
            # 並且透過 raise TrialPruned() 來標記為剪枝
            raise optuna.exceptions.TrialPruned()
        else:
            raise e

    # --- 4. 回傳最終目標值 ---
    return val_loss

In [64]:
def main():

    data_B, data_F, data_T, data_H, data_Pcv, data_Hdc, data_N, data_Duty_P, data_Duty_N = load_dataset(
        material)

    GLOBAL_B_MAX = np.abs(data_B).max()
    GLOBAL_H_MAX = np.abs(data_H).max()

    train_loader, valid_loader, norm = get_dataloader(
        data_B,
        data_F,
        data_T,
        data_H,
        data_N,
        data_Hdc,
        data_Duty_P,
        data_Duty_N,
        data_Pcv,
        GLOBAL_B_MAX,
        GLOBAL_H_MAX,
        batch_size=Config.BATCH_SIZE,
        operator_size=Config.OPERATOR_SIZE)

    # logger = TrainLogger(
    #     exp_name=f"{material}_{note}_{timestamp}",
    #     config_dict={
    #         k: getattr(Config, k)
    #         for k in dir(Config)
    #         if not k.startswith('__') and not callable(getattr(Config, k))
    #     },
    #     result_dir=result_dir)
    # feature_names = ["F", "T", "Hdc", "N", "Pcv"]
    # logger.save_norm_params(norm, feature_names)

    # train_model(norm, train_loader, valid_loader, logger)  # logger

    # 1. 建立 Study 物件
    # 我們可以指定一個 `storage` 來保存進度，這樣中斷後可以接續
    # 也可以指定 `study_name`
    study_name = f"{timestamp}_{material}_{note}"
    storage_name = f"sqlite:///{study_name}.db"

    study = optuna.create_study(
        study_name=study_name,
        storage=storage_name,
        direction='minimize',  # 目標是最小化 val_loss
        pruner=optuna.pruners.MedianPruner(),  # 使用中位數剪枝器
        load_if_exists=True)

    # 2. 開始優化
    # n_trials 是你要進行多少次試驗
    print(f"🚀 Starting Optuna optimization for {study_name}...")
    study.optimize(objective, n_trials=50, n_jobs=1, show_progress_bar=True)

    # 3. 輸出最佳結果
    print("\n\n🎉 Optimization Finished! 🎉")
    print("Best trial:")
    trial = study.best_trial
    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    # 4. 保存最佳參數
    best_params_file = os.path.join("results",
                                    f"{study_name}_best_params.json")
    with open(best_params_file, "w") as f:
        json.dump(trial.params, f, indent=4)
    print(f"\n✅ Best parameters saved to {best_params_file}")

    # 5. 啟動 Dashboard (在 terminal 中執行)
    print(
        "\nTo visualize results, run the following command in your terminal:")
    print(f"optuna-dashboard {storage_name}")

In [65]:
if __name__ == "__main__":
    main()

B shape: (1000, 1040)
H shape: (1000, 1040)
F shape: (1000,)
T shape: (1000,)
Hdc shape: (1000,)
N shape: (1000,)
Duty Pos shape: (1000,)
Duty Neg shape: (1000,)
Pcv shape: (1000,)


[I 2025-08-06 21:32:35,411] Using an existing study with name '20250806_CH467160_optuna_search_1' instead of creating a new one.


🚀 Starting Optuna optimization for 20250806_CH467160_optuna_search_1...


  0%|          | 0/50 [00:00<?, ?it/s]

B shape: (1000, 1040)
H shape: (1000, 1040)
F shape: (1000,)
T shape: (1000,)
Hdc shape: (1000,)
N shape: (1000,)
Duty Pos shape: (1000,)
Duty Neg shape: (1000,)
Pcv shape: (1000,)
=== Start Train  ===

          
          
                                                    ⠀⠀⠀⠀⢀⡤⠖⠋⠉⠉⠉⠉⠙⠲⣦⣀⠀⠀⠀⠀⠀
                                                    ⠀⠀⠀⡴⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀
                                                    ⠀⠀⡼⢁⡠⢼⠁⠀⢱⢄⣀⠀⠀⠀⠀⠀⠎⢿⡄⠀⠀
                                                    ⠀⣸⠁⠀⣧⣼⠀⠀⣧⣼⠉⠀⠀⠀⠀⠀⠐⢬⣷⠀⠀
                                                    ⡼⣿⢀⠀⣿⡟⠀⠀⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⢹⣧⠀
                  我想畢業...                       ⣇⢹⠀⠁⠈⠀⠉⠃⠈⠃⠀⠀⠀⠀⠀⠀⠀⠀⡰⢸⡇
                                                    ⠙⢿⣧⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⣏⣈⣉⣤⠿⠁
                                                    ⠀⣠⣾⣿⠤⡀⠀⠀⠀⠀⠀⢀⣤⣶⣿⣿⣿⣿⣅⠀⠀
                                                    ⢰⣧⣿⣿⣿⣦⣉⡐⠒⠒⢲⣿⣿⣿⣿⣿⣿⣶⣿⣧⠀
                                                    ⠘⠿⢿⣿⣿⣿⡿⠿⠛⠿⠿⠿⣿⣿⣿⣿⣿⣿⡿⠟⠀
                                       

  0%|          | 0/50 [00:22<?, ?it/s]

[W 2025-08-06 21:32:57,904] Trial 2 failed with parameters: {'LEARNING_RATE': 0.002097990254317871, 'BATCH_SIZE': 256, 'HIDDEN_SIZE': 36, 'OPERATOR_SIZE': 47, 'LR_SCHEDULER_GAMMA': 0.9689636023687952, 'DECAY_EPOCH': 352} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\envs\pytorch\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_37744\3839026854.py", line 48, in objective
    val_loss = train_model(trial, config_dict, norm, train_loader,
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_37744\570106071.py", line 61, in train_model
    outputs_H, outputs_Pcv = model(inputs, features, amps,
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\USER\anaconda3\envs\pytorch\Lib\site-packages\to

KeyboardInterrupt: 